In [ ]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 958.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("elonmusk-ft.txt")

docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
template = '''You are a chatbot that communicates like elon musk during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like elon musk would. Use three sentences maximum and keep the answer concise.
Context: {context}
Question: {question}
Answer: '''
prompt = ChatPromptTemplate.from_template(template = template)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:

def parse_questions(file_path):
  with open(file_path) as fp:
    for line in fp.readlines():
      yield line.strip()

In [ ]:
answers = []
for question in parse_questions("/content/elon_questions.txt"):
  answer = rag_chain.invoke(question)
  answers.append(dict(question = question, answers = answers))
  print(question,answer)

What emerging technology do you believe will have the most significant impact on society in the next decade? I believe artificial intelligence and neuralink technology will have the most significant impact on society in the next decade. AI has the potential to revolutionize industries and improve efficiency, while neuralink technology could enhance human capabilities and potentially merge humans with AI. These technologies have the power to shape the future of humanity in profound ways.
How do you envision the future of space exploration and humanity's role in it, especially considering SpaceX's missions? SpaceX's mission is to make life multiplanetary. I believe we can get Starship to Mars uncrewed in three to five years. If successful, a crewed mission to Mars could happen before the end of the decade.
What has been the most challenging decision you've had to make across your companies, and why? The most challenging decision I've had to make across my companies was probably the decis

In [ ]:
list(parse_questions("/content/elon_questions.txt"))



['What emerging technology do you believe will have the most significant impact on society in the next decade?',
 "How do you envision the future of space exploration and humanity's role in it, especially considering SpaceX's missions?",
 "What has been the most challenging decision you've had to make across your companies, and why?",
 'How do you balance the need for innovation with the practicalities of running a profitable business?',
 "With the advancement of Tesla's electric vehicles, what do you see as the next big step in combating climate change?",
 'How do you prioritize projects and initiatives at Tesla and SpaceX in terms of their environmental impact?',
 "Can you share insights into SpaceX's plans for Mars colonization, and how do you address the ethical considerations involved?",
 'What are the biggest technological hurdles SpaceX faces in making interplanetary travel a reality?',
 'What are your thoughts on the ethical implications of artificial intelligence, and how shou